<h1><center>IAGA Summer School 2019</center></h1>

<h1><center>Visualising geomagnetic data: time series, Bartels plots and contour plots</center></h1>

In [ ]:
# Import notebook dependencies

import os
import sys
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from scipy.interpolate import griddata

sys.path.append('..')
from src import mag_lib as mag

# Set the data path

ESK_2003_PATH = os.path.abspath('../data/external/ESK_2003/')

In [ ]:
%matplotlib notebook

# Plotting 1-minute and 1-hour observatory means

Load the 2003 1-minute data for ESK and resample to hourly means

In [ ]:
# 1-minute data
obs_min = mag.load_year(observatory='esk', year=2003, path=ESK_2003_PATH)
# 1-hour data
obs_hourly = obs_min.resample('1h').mean()

Print the first few lines of each data set

In [ ]:
obs_min.head()

In [ ]:
obs_hourly.head()

### >> USER INPUT HERE: Choose the start and end times of the time series plots in the format '2003-mm-dd hh'

In [ ]:
start_date = '2003-01-01 00'
end_date = '2003-01-31 23'

Plot the 1-minute data for the selected period

In [ ]:
ax = obs_min.loc[start_date:end_date, 'ESKX':'ESKZ'].plot(subplots=True, figsize=(10,7),
                                                          title='1-minute means: ESK 2003', legend=False)
plt.xlabel('Date')
ax[0].set_ylabel('X (nT)')
ax[1].set_ylabel('Y (nT)')
ax[2].set_ylabel('Z (nT)')

Plot the hourly mean data for the same period

In [ ]:
ax = obs_hourly.loc[start_date:end_date, 'ESKX':'ESKZ'].plot(subplots=True, figsize=(10,7),
                                                             title='1-hour means: ESK 2003', legend=False)
plt.xlabel('Date')
ax[0].set_ylabel('X (nT)')
ax[1].set_ylabel('Y (nT)')
ax[2].set_ylabel('Z (nT)')

Compare hourly means with the annual means

In [ ]:
# Resample to annual means
obs_annual = obs_min.resample('1Y').mean()
dates = obs_hourly.loc[start_date:end_date].reset_index()['DATE_TIME']
annual = pd.DataFrame(index=dates, data=[[obs_annual.loc['2003-12-31','ESKX'], obs_annual.loc['2003-12-31','ESKY'],
                                       obs_annual.loc['2003-12-31','ESKZ']]], columns=['ESKX', 'ESKY', 'ESKZ'])

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, sharex=True, figsize=(10,9))
# X component
ax1.plot(dates, obs_hourly.loc[start_date:end_date,'ESKX'], 'k')
ax1.plot(annual['ESKX'], 'k')
ax1.fill_between(dates, obs_hourly.loc[start_date:end_date,'ESKX'], annual['ESKX'].values,
                 where=obs_hourly.loc[start_date:end_date,'ESKX'] < annual['ESKX'], facecolor='lightblue',
                 interpolate=True)
ax1.fill_between(dates, obs_hourly.loc[start_date:end_date,'ESKX'], annual['ESKX'],
                 where=obs_hourly.loc[start_date:end_date,'ESKX'] >= annual['ESKX'], facecolor='pink',
                 interpolate=True)
ax1.set_ylabel('X (nT)')
# Y component
ax2.plot(dates, obs_hourly.loc[start_date:end_date,'ESKY'], 'k')
ax2.plot(annual['ESKY'], 'k')
ax2.fill_between(dates, obs_hourly.loc[start_date:end_date,'ESKY'], annual['ESKY'].values,
                where=obs_hourly.loc[start_date:end_date,'ESKY'] < annual['ESKY'], facecolor='lightblue',
                 interpolate=True)
ax2.fill_between(dates, obs_hourly.loc[start_date:end_date,'ESKY'], annual['ESKY'],
                 where=obs_hourly.loc[start_date:end_date,'ESKY'] >= annual['ESKY'], facecolor='pink',
                 interpolate=True)
ax2.set_ylabel('Y (nT)')
# Z component
ax3.plot(dates, obs_hourly.loc[start_date:end_date,'ESKZ'], 'k')
ax3.plot(annual['ESKZ'], 'k')
ax3.fill_between(dates, obs_hourly.loc[start_date:end_date,'ESKZ'], annual['ESKZ'].values,
                where=obs_hourly.loc[start_date:end_date,'ESKZ'] < annual['ESKZ'], facecolor='lightblue',
                 interpolate=True)
ax3.fill_between(dates, obs_hourly.loc[start_date:end_date,'ESKZ'], annual['ESKZ'],
                where=obs_hourly.loc[start_date:end_date,'ESKZ'] >= annual['ESKZ'], facecolor='pink',
                 interpolate=True)
ax3.set_ylabel('Z (nT)')
plt.xticks(rotation = 45)
plt.xlabel('Date', )

# Data stacking

Stacking is a simple technique for signal to noise improvement in a time series. If a signal has a period $T$ then dividing the signal into sections of length $T$ and taking a mean over many stacked sections should reinforce the signal and reduce the amplitude of random noise.

In [ ]:
nsegs = 10
nmins = 1440
ntot = nsegs*nmins-1
nroll = 1
starting_day = '2003-06-01'

In [ ]:
def read_obs_hmv_d(obscode, year_st, year_fn, folder):
    """Read (or calculate) the declination hourly means between year_st and year_fn for the specified observatory
    
    """
    OBSY   = obscode.upper()
    obsy   = obscode.lower()
    # Read in the observatory data one year file at a time and construct filenames
    datareq = pd.DataFrame()
    for year in range(year_st, year_fn+1):
        ystr    = str(year)
        file    = obsy + ystr + 'dhor.hor'
        fpf     =  folder + file
        tmp     = mag.IAGA2002_Data_Reader(fpf)
        tmp.columns = [col.strip(OBSY) for col in tmp.columns]
        if('D' not in tmp.columns):
            dvals, hvals, ivalsm, fvals  = mag.xyz2dhif(tmp['X'], tmp['Y'], tmp['Z'])
            tmp.insert(1, dvals)
        datareq = datareq.append(tmp[['D']])
    return(datareq)


In [ ]:
# Read data
hmv = read_obs_hmv_d(obscode='esk', year_st=1986, year_fn=1997, folder='/Users/gracecox/Dropbox/David_IAGA/hmv/')

hmv.reset_index(inplace=True)
# Set missing values to nan
print(99999.00 in hmv.D.values)
hmv['D']=hmv.D.replace(99999.00, np.nan)
print(99999.00 in hmv.D.values)


In [ ]:
# Calculate the time as minutes since midnight and date as julian day or a time delta in seconds
# This is needed for plotting purposes as the axes must be numbers, not datetime objects or strings
hmv['mins_count'] = hmv['DATE_TIME'].map(lambda x: x.time().hour*60 + x.time().minute)

hmv['julian_day'] = hmv['DATE_TIME'].map(lambda x: int(x.to_julian_date()))

hmv['seconds'] = pd.to_timedelta(hmv.DATE_TIME).dt.total_seconds().astype(int)

In [ ]:
print(np.min(df['D']))


In [ ]:
# Plot the hourly data in 2D using a dataframe pivot and contourf
df = pd.DataFrame({'date':hmv['julian_day'], 'time':hmv['mins_count'], 'D':hmv['D']})

dfMesh = df.pivot('time','date','D')

fig, ax = plt.subplots(figsize=(8,5))
#v = np.linspace(-500, -250, 100, endpoint=True)
conts = ax.contourf(dfMesh.columns.values, dfMesh.index.values, dfMesh.values, cmap=cm.jet)
ax.set_xlabel('Julian Day')
ax.set_ylabel('Minutes since midnight')
#plt.colorbar()
plt.show()

In [ ]:
# Plot the hourly data in 3D using trisurf
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.plot_trisurf(hmv.seconds, hmv.mins_count, hmv.D)
xlabels=ax.get_xticks().tolist()
ax.set_xticklabels(pd.to_datetime(xlabels, unit='s'))
ylabels=ax.get_yticks()

In [ ]:
# Subtract the daily mean from each hour
hmv['daily'] = hmv['D'] - np.array(hmv.set_index('DATE_TIME').groupby(pd.Grouper(freq='D')).mean()['D']).repeat(24)

In [ ]:
# Plot the hourly data (with daily mean subtracted) in 3D using trisurf
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.plot_trisurf(hmv.seconds, hmv.mins_count, hmv.daily)
xlabels=ax.get_xticks().tolist()
ax.set_xticklabels(pd.to_datetime(xlabels, unit='s'))
ylabels=ax.get_yticks()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

## Matplotlib Sample Code using 2D arrays via meshgrid
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X ** 2 + Y ** 2)
Z = np.sin(R)
fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_zlim(-1.01, 1.01)

ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

fig.colorbar(surf, shrink=0.5, aspect=5)
plt.title('Original Code')
plt.show()

In [ ]:
np.shape(Z)

In [ ]:
x2, y2 = np.meshgrid(np.array(hmv['seconds'][0:1000]), np.array(hmv['mins_count'][0:1000]))

# Interpolate unstructured D-dimensional data.
z2 = griddata((hmv['seconds'][0:1000], hmv['mins_count'][0:1000]), hmv['daily'][0:1000], (x2, y2), method='cubic')

# Ready to plot
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x2, y2, z2, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_zlim(-1.01, 1.01)

ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

fig.colorbar(surf, shrink=0.5, aspect=5)
plt.title('Meshgrid Created from 3 1D Arrays')

plt.show()

In [ ]:
X = np.array(hmv['seconds'][0:1000])
Y = np.array(hmv['mins_count'][0:1000])
X, Y = np.meshgrid(X, Y)
zs = np.array(np.ravel(X), np.ravel(Y))
Z = zs.reshape(X.shape)
#Z = np.array(hmv['daily'][0:1000])
fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
#ax.set_zlim(-1.01, 1.01)

#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

#fig.colorbar(surf, shrink=0.5, aspect=5)
plt.title('Original Code')
plt.show()

In [ ]:
# Plot the hourly data with daily mean subtracted in 2D using a dataframe pivot and contourf
df = pd.DataFrame({'date':hmv['julian_day'], 'time':hmv['mins_count'], 'D':hmv['daily']})

dfMesh = df.pivot('time','date','D')

fig, ax = plt.subplots(figsize=(8,5))

conts = ax.contourf(dfMesh.columns.values, dfMesh.index.values, dfMesh.values,100, cmap=cm.jet)
ax.set_xlabel('Julian Day')
ax.set_ylabel('Minutes since midnight')
#plt.colorbar()
plt.show()

In [ ]:
hmv

In [ ]:
# Calculate the monthly mean for each hour of the day
mmv = hmv.groupby([hmv['DATE_TIME'].dt.year, hmv['DATE_TIME'].dt.hour]).mean()
                                        

In [ ]:
mmv

In [ ]:
# Plot the monthly mean of each hour using trisurf
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.plot_trisurf(mmv.seconds, mmv.mins_count, mmv.D)
xlabels=ax.get_xticks().tolist()
ax.set_xticklabels(pd.to_datetime(xlabels, unit='s'))
ylabels=ax.get_yticks()
#ax.set_yticklabels((ylabels-30)/60)

In [ ]:
xlabels

In [ ]:
pd.to_datetime(xlabels, unit='s')

In [ ]:
pd.to_datetime(hmv.julian_day, unit='D')

In [ ]:
np.array(hmv.groupby(pd.Grouper(freq='A')).mean()).repeat(12)

In [ ]:
mmv

In [ ]:
mmv.columns

In [ ]:
hmv.groupby(pd.Grouper(freq='D'))

In [ ]:
df.set_index(['year', 'month', 'item']).unstack(level=-1)

In [ ]:
mmv.unstack(level=-1)

In [ ]:
dfMesh

In [ ]:
df

In [ ]:
df = pd.DataFrame({'date':hmv['julian_day'], 'time':hmv['mins_count'], 'D':hmv['D']})

dfMesh = df.pivot('time','date','D')

fig, ax = plt.subplots()

conts = ax.contourf(dfMesh.columns.values, dfMesh.index.values, dfMesh.values,100, cmap=cm.jet)
ax.set_xlabel('Julian Day')
ax.set_ylabel('Minutes since midnight')
#plt.colorbar()
plt.show()

In [ ]:
dfMesh = hmv.pivot('mins_count','julian_day','X')

In [ ]:
import matplotlib.dates as dates
hmv['date_num'] = hmv['DATE_TIME'].map(lambda x: dates.date2num(x))

In [ ]:
 ([df1.index.year, df1.index.hour])

In [ ]:

test = np.array(hmv.set_index('DATE_TIME').groupby(
    pd.Grouper(freq='M')).mean()) - np.array(hmv.set_index('DATE_TIME').groupby(
    pd.Grouper(freq='Y')).mean()).repeat(12)



In [ ]:
plt.plot(test)

In [ ]:
help(np.meshgrid)

In [ ]:
x = np.array(hmv['seconds'])
y = np.array(hmv['mins_count'])

In [ ]:
X, Y = np.meshgrid(x,y)
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')
surf1 = ax.plot_surface(dfMesh.columns.values, dfMesh.index.values, dfMesh.values, cmap=plt.cm.coolwarm)

In [ ]:
dfMesh

In [ ]:
#ts = pd.date_range('1/1/2017 0:00', '1/24/2017 23:00', freq='H') # 24 * 24 long
#temp = map(lambda x: sin(2*pi*x/40), range(576))  
# tiny testcase: sin(2*pi*x/12) or /24 provide horizontal contours: quite right.


df = pd.DataFrame({'date':map(lambda x:int(x.to_julian_date()), hmv['DATE_TIME']),
                   'time':map(lambda x:x.time().hour*60 + x.time().minute, hmv['DATE_TIME']),
                   'X':hmv['X']})

#dfMesh = df.pivot('time','date',hmv['X'])

#fig, ax = plt.subplots()

#conts = ax.contour(dfMesh.columns.values, dfMesh.index.values, dfMesh.values)
#ax.set_xlabel('Julian day')
#ax.set_ylabel('Minutes since midnight')

#plt.show()

# Plotting observatory data by the Bartels rotation number

Demo of using pandas & matplotlib to reproduce yearly plots of observatory data ordered by Bartels rotation number

e.g. http://geomag.bgs.ac.uk/data_service/data/bulletins/esk/2003/esk_dec03.pdf pages 22-24

**Load 1-minute data and resample to 1-hour**

In [ ]:
# 1-minute data
df_m = mag.load_year(observatory='esk', year=2003, path=ESK_2003_PATH)
# 1-hour data
df_h = df_m.resample('1h').mean()
df_h.head()

**Calculate hourly means and append them to the hourly dataframe**

In [ ]:
obs = 'ESK'
annual_means = (
    df_m[[f"{obs}{i}" for i in "XYZF"]].resample("1Y").mean()
    .rename(columns={f"{obs}{i}":f"{obs}{i}_mean" for i in "XYZF"})
    .reindex(index=df_h.index, method="nearest")
)
df_h = df_h.join(annual_means)
df_h.head()

**Change the index to a Bartels-rotation-based MultiIndex**

In [ ]:
def bartels_rotation(datetime, return_indexes=True):
    """
    
    Args:
        date (datetime/DatetimeIndex)
        
    Returns:
        tuple (rotation number, day within rotation, hour within day)
        
    """
    if type(datetime) is pd.DatetimeIndex:
        date, hour = datetime.date, datetime.hour
    elif type(datetime) is dt.datetime:
        date, hour = datetime.date(), datetime.hour
    # Number of days since Bartels 0
    ndays = pd.to_timedelta(date - dt.date(1832, 2, 8)).days
    bartels_rotation = ndays//27 + 1
    day = ndays%27 + 1
    if return_indexes:
        return bartels_rotation, day, hour
    else:
        return tuple([item.values for item in (bartels_rotation, day, hour)])


def create_bartels_index(df, nlevels=3):
    """Replaces a DatetimeIndex with a Bartels MultiIndex
    
    If nlevels is 3, the MultiIndex has levels:
        0: (bartrot) Bartel rotation number
        1: (bartrotday) Day within the current Bartel rotation
        2: (hourofday) Hour within the current day
    else if nlevels is 2:
        0: (bartrot) Bartel rotation number
        1: (bartrotday) Fractional day
    
    Args:
        df (DataFrame)
    
    Returns:
        DataFrame

    """
    bartrot, bartrotday, hourofday = bartels_rotation(df.index)
    if nlevels == 3:
        df.index = pd.MultiIndex.from_arrays(
            (bartrot, bartrotday, hourofday), names=("bartrot", "bartrotday", "hourofday")
        )
    elif nlevels == 2:
        df.index = pd.MultiIndex.from_arrays(
            (bartrot, bartrotday + hourofday/24), names=("bartrot", "bartrotday")
        )
    else:
        raise NotImplementedError()
    return df

In [ ]:
# Preserve the time index as a column instead
df_h["time"] = df_h.index
df_h = create_bartels_index(df_h, nlevels=2)

df_h.head()

**Identifying calendar month starts**

In [ ]:
month_starts = df_h["time"].where(
    (df_h["time"].dt.day == 1) & (df_h["time"].dt.hour == 0)
).dropna()
month_starts

In [ ]:
month_starts.loc[2312].dt.strftime("%b").values[0]

**Loop through each bartrot and plot it on its own axis**

In [ ]:
obs = "ESK"
var = "X"

bartrots = range(df_h.index[0][0], df_h.index[-1][0] + 1)
fig, axes = plt.subplots(
    nrows=len(bartrots), ncols=1, figsize=(15, 15),
    gridspec_kw = {'hspace':0},
    sharex=True, sharey=True
)
for bartrot, ax in zip(bartrots, axes):
    x = df_h.loc[bartrot].index
    y0 = df_h.loc[bartrot][f"{obs}{var}_mean"]
    y1 = df_h.loc[bartrot][f"{obs}{var}"]
    ax.plot(x, y0, color="black", linewidth=0.4)
    ax.plot(x, y1, color="black", linewidth=0.8, clip_on=False)
    ax.fill_between(
        x, y0, y1, where=y1 < y0,
        facecolor='lightblue', interpolate=True, zorder=9#, clip_on=False currently causes a bug
    )
    ax.fill_between(
        x, y0, y1, where=y1 >= y0,
        facecolor='pink', interpolate=True, zorder=10#, clip_on=False
    )
    ax_r = ax.twinx()
    ax_r.set_ylabel(bartrot, fontsize=15)
    ax_r.set_yticks([])
    # some magic which enables lines from one axis to show on top of other axes
    ax.patch.set_visible(False)
    
    # Add text identifying the start of each month
    if bartrot in month_starts.keys():
        bartrotday = float(month_starts.loc[bartrot].index.values)
        month = month_starts.loc[bartrot].dt.strftime("%b").values[0]
        ax.text(bartrotday, y0.iloc[0] - 85, month, verticalalignment="top")

ax.set_ylim((y0.iloc[0] - 75, y0.iloc[0] + 75))
ax.set_xlabel("Day within Bartels rotation");
axes[0].set_title(f"{obs}-{var} (nT, left axis), by Bartels rotation number (right axis)",
                  fontsize=15);